In [69]:
import pandas as pd
import os
from datetime import datetime
import time

In [70]:
# START SCRIPT TIMER
print(f"Start Time: {datetime.now()}")
tic = time.perf_counter()

Start Time: 2024-04-16 20:36:58.672042


In [71]:
# ADD USER FILE PATHS
dunslocs_file = r'Z:\UHC_Data\NETS_UHC\NETS2022\Geodatabases\NETS2022_locs.gdb\DunsLocations20231130_v2'
mesa_locs_file = r'X:\AddressGeocoding\From_MESA_Air\Data\MESAAIR_locs.gdb\mesa_locs_aeac_zcta10'
near_table_file = r'X:\AddressGeocoding\From_MESA_Air\Data\Temp\scratch\NETS_linkage_test.gdb\mesa_nets_linkage_test'

classifiedlong_file = r'Z:\UHC_Data\NETS_UHC\NETS2022\Data\Final\ClassifiedLong20231127.txt'
dunsmove_file = r'Z:\UHC_Data\NETS_UHC\NETS2022\Data\Final\DunsMove20231201.txt'
cat_descriptions_file = r'Z:\UHC_Data\NETS_UHC\NETS2022\Data\Final\CategoryDescriptions20231127.txt'
# this is either the entire dunslocation file or a subset of the file that is linked to participants
dunslocation_file = r'Z:\UHC_Data\NETS_UHC\NETS2022\Data\Final\DunsLocation20231207.txt'
xwalk_file = r'Z:\UHC_Data\NETS_UHC\NETS2022\Data\Final\BG_CC_TC_Xwalk20231023.txt'
output_folder = r'D:\scratch'

In [72]:
arcpy.env.workspace = r"F:\Arc_Projects\NETS_test_linkage"

In [73]:
#%% CONVERT FC TABLE TO PANDAS DF

# define function to convert fc table to pandas dataframe
def table_to_data_frame(in_table, input_fields=None, where_clause=None):
    """Function will convert an arcgis table into a pandas dataframe with an object ID index, and the selected
    input fields using an arcpy.da.SearchCursor."""
    OIDFieldName = arcpy.Describe(in_table).OIDFieldName
    if input_fields:
        final_fields = [OIDFieldName] + input_fields
    else:
        final_fields = [field.name for field in arcpy.ListFields(in_table)]
    data = [row for row in arcpy.da.SearchCursor(in_table, final_fields, where_clause=where_clause)]
    fc_dataframe = pd.DataFrame(data, columns=final_fields)
    fc_dataframe = fc_dataframe.set_index(OIDFieldName, drop=True)
    return fc_dataframe

In [74]:
# LOAD NEAR TABLE AND MESA LOCS THEN MERGE
near_df = table_to_data_frame(near_table_file, input_fields=['IN_FID', 'NEAR_FID', 'NEAR_DIST'])
mesa_locs = table_to_data_frame(mesa_locs_file, input_fields=['LOCID_DREXEL', 'UHCMatchCodeRank'])

join_locid = (near_df
              .merge(mesa_locs, how='left', left_on='IN_FID', right_on='OBJECTID')
              .drop(columns=['IN_FID'])
              .rename(columns={'UHCMatchCodeRank': 'UHCMatchCodeRank_MESA'})
             )

In [75]:
dunslocs = table_to_data_frame(dunslocs_file, input_fields=['AddressId', 'UHCMatchCodeRank'])
join_dunslocid = (join_locid
                  .merge(dunslocs, how='left', left_on='NEAR_FID', right_on='OBJECTID')
                  .drop(columns=['NEAR_FID'])
                  .rename(columns={'UHCMatchCodeRank': 'UHCMatchCodeRank_NETS'})
                 )

In [76]:
# END SCRIPT TIMER 
toc = time.perf_counter()
t = toc - tic
print(f'total time: {round(t/60, 2)} minutes')

total time: 20.68 minutes


In [80]:
join_dunslocid.shape

(242424133, 5)